# Lab 5 - Implementation of the Transportation Simplex Method

<b>Information on group members:</b><br>
1) ER-2068, Eka Tsilosani <br>
2) ER-2067, Temur Tsomaia

## Instruction
Use the Python programming language. The problem should be represented using some matrix form, not a net. You can use the
example problems provided in this script when implementing the method. Then, analyze the method’s performance using
different problem sizes (e.g., m, n = 5, 10, 15, average results over, e.g., 10 runs). Verify how quickly, i.e., in how many
iterations, the method reaches the optimum. You should provide some additional data on, e.g., average chain sizes
constructed via the Transportation Method. Construct the test samples randomly (e.g., cost, supply, demand = rand(1,
10)), but let them occasionally contain zeroes, M-values, and even 0-rows or columns. Note that the total supply should
equal the total demand to allow the problem to have feasible solutions. Report the results in your script (it is preferable to
use the Jupyter Notebook). Important: verify the correctness of your implementation by applying the PuLP library to solve
the problem and compare obtained solutions in terms of the objective function. Note that in this class of problems, it is
common that there are multiple optima to the problem. Hence, the comparison with the oracle (PuLP) can be based only
on comparing the final objective function values (should be equal).

### Block 1: Initial Setup, Imports and Data Generation

The initial setup defines the computational environment and the scope of testing. The constant $\text{BIG\_M}$ ($10^9$) is employed as a standard linear programming technique to model prohibited routes in the cost matrix, ensuring the solver correctly avoids non-feasible links. The constant $\text{EPSILON}$ ($10^{-9}$) manages numerical stability, being essential for precision when checking for zero values in flow allocations and managing the boundary conditions of degeneracy.

The generate_random_problem(m, n) function is critical as the data source, guaranteeing that every test case maintains supply-demand balance ($\sum \text{Supply} = \sum \text{Demand}$) to ensure solution feasibility. This generator systematically introduces complexity by including zero-cost routes, $\text{BIG\_M}$ penalties, and occasionally resetting entire rows or columns to zero, providing a comprehensive test environment.

The solve_with_pulp utility serves as the Validation Oracle, independently solving each generated problem using the industry-standard PuLP library. The objective value returned by PuLP establishes the true minimum total cost, allowing for direct comparison. This methodology is vital because, due to the structure of transportation problems, multiple basic feasible solutions may yield the same optimal cost, making the objective function value the only reliable metric for correctness.

In [13]:
import numpy as np
import pandas as pd
import time
import pulp
from copy import deepcopy
import itertools

# Constant for Big M value (for prohibited routes)
BIG_M = 1e9
# Epsilon for degeneracy handling and float precision
EPSILON = 1e-9


# DATA GENERATION FUNCTION

def generate_random_problem(m, n, max_cost=100, max_supply=1000):

    # Generates a random transportation problem (m x n) ensuring Total Supply = Total Demand.

    # 1. Random Costs generation
    costs = np.random.randint(1, max_cost, size=(m, n)).astype(float)
    mask_zero = np.random.rand(m, n) < 0.05
    costs[mask_zero] = 0.0
    mask_m = np.random.rand(m, n) < 0.01
    costs[mask_m] = BIG_M

    # 2. Supply and Demand generation
    supply = np.random.randint(10, max_supply, size=m).astype(float)
    demand = np.random.randint(10, max_supply, size=n).astype(float)

    # 3. Balancing (Total Supply == Total Demand)
    diff = supply.sum() - demand.sum()
    if abs(diff) > 1e-4:
        if diff > 0:
            demand[np.random.randint(0, n)] += diff
        else:
            supply[np.random.randint(0, m)] += abs(diff)

    # Introduce occasional zero-rows or columns
    if np.random.rand() < 0.05 and m > 1:
        costs[np.random.randint(0, m), :] = 0.0
    if np.random.rand() < 0.05 and n > 1:
        costs[:, np.random.randint(0, n)] = 0.0

    return supply, demand, costs


# VALIDATION FUNCTION USING PULP (The Oracle)

def solve_with_pulp(supply, demand, costs):

    # Solves the transportation problem using PuLP for verification.

    m, n = costs.shape
    prob = pulp.LpProblem("Transportation_Verification", pulp.LpMinimize)
    x = pulp.LpVariable.dicts("Route", (range(m), range(n)), lowBound=0, cat='Continuous')

    prob += pulp.lpSum([x[i][j] * costs[i, j] for i in range(m) for j in range(n)])

    for i in range(m):
        prob += pulp.lpSum([x[i][j] for j in range(n)]) == supply[i], f"Supply_Constraint_{i}"
    for j in range(n):
        prob += pulp.lpSum([x[i][j] for i in range(m)]) == demand[j], f"Demand_Constraint_{j}"

    prob.solve(pulp.PULP_CBC_CMD(msg=False, options=['presolve off', 'primalToler 1e-6']))

    if pulp.LpStatus[prob.status] == "Optimal":
        return pulp.value(prob.objective)
    return np.inf

### Block 2: TransportationSimplex Class (NWC + Float Fixes)

The TransportationSimplex class encapsulates the entire optimization algorithm.

The solution process begins with finding an Initial Basic Feasible Solution (IBFS) using the North-West Corner (NWC) rule, implemented in _get_initial_solution_nwc(). This step is designed for robustness: when both supply and demand for a cell are simultaneously depleted (degeneracy), the algorithm breaks the tie by advancing only one index, explicitly ensuring that the basis maintains exactly $m+n-1$ cells, a critical structural requirement.

The algorithm then enters its main iterative loop, employing the Modified Distribution (MODI) method to perform the optimality check. The _calculate_potentials() function solves for the dual variables, or potentials ($u_i$ for rows and $v_j$ for columns), using the governing equation $u_i + v_j = c_{ij}$ exclusively across the current basic cells. These potentials are then utilized to calculate the reduced cost for every non-basic cell using the formula: $\bar{c}_{ij} = c_{ij} - (u_i + v_j)$. Optimality is achieved when all reduced costs are non-negative ($\bar{c}_{ij} \ge 0$).

If the solution is not optimal, the cell with the most negative reduced cost is selected as the entering variable. The _find_cycle(start\_pos) method, using a Depth-First Search (DFS) methodology, traces the unique closed loop formed by the entering cell and the existing basic variables. The pivoting operation calculates $\Theta$ (Theta), the minimum allocation found on the negative-signed nodes of the cycle, which dictates the maximum feasible flow shift. The flows are updated by adding and subtracting $\Theta$ around the cycle, and the cell that first reaches zero flow becomes the leaving variable, completing the basis exchange and advancing to a better basic feasible solution.

In [14]:
class TransportationSimplex:
    def __init__(self, supply, demand, costs):
        self.supply = supply.copy()
        self.demand = demand.copy()
        self.costs = costs.copy()
        self.m, self.n = self.costs.shape
        self.allocation = np.zeros((self.m, self.n))
        self.basis = np.zeros((self.m, self.n), dtype=bool)

        self.iterations = 0
        self.chain_lengths = []

    def _get_initial_solution_nwc(self):

        # DIAGNOSTIC IBFS: North-West Corner Rule.
        # Uses EPSILON for robust degeneracy handling.

        i, j = 0, 0
        current_supply = self.supply.copy()
        current_demand = self.demand.copy()

        self.allocation.fill(0)
        self.basis.fill(False)

        while i < self.m and j < self.n:
            quantity = min(current_supply[i], current_demand[j])
            self.allocation[i, j] = quantity
            self.basis[i, j] = True

            current_supply[i] -= quantity
            current_demand[j] -= quantity

            # Check exhaustion with EPSILON
            row_exhausted = abs(current_supply[i]) < EPSILON
            col_exhausted = abs(current_demand[j]) < EPSILON

            if row_exhausted and col_exhausted:
                # Degeneracy: If double zero, advance only one index (j), keeping basis true
                j += 1
            elif row_exhausted:
                i += 1
            elif col_exhausted:
                j += 1


    def _calculate_potentials(self):

        # OPTIMIZED: Calculates u and v using MODI method (Queue-based).

        u = np.full(self.m, np.nan)
        v = np.full(self.n, np.nan)
        u[0] = 0.0

        queue = [(0, 0)]

        while queue:
            type_id, index = queue.pop(0)

            if type_id == 0:
                for j in np.where(self.basis[index, :])[0]:
                    if np.isnan(v[j]):
                        v[j] = self.costs[index, j] - u[index]
                        queue.append((1, j))

            else:
                for i in np.where(self.basis[:, index])[0]:
                    if np.isnan(u[i]):
                        u[i] = self.costs[i, index] - v[index]
                        queue.append((0, i))

        u[np.isnan(u)] = 0
        v[np.isnan(v)] = 0

        return u, v

    def _find_cycle(self, start_pos):

        # Finds the unique closed loop using Depth-First Search (DFS).

        start_r, start_c = start_pos

        def dfs(curr_r, curr_c, path, direction):
            path.append((curr_r, curr_c))

            if len(path) > 3 and curr_r == start_r and curr_c == start_c:
                return path

            if direction == 0: # Search Horizontally (rows)
                for j in range(self.n):
                    is_valid_move = self.basis[curr_r, j] or (curr_r, j) == start_pos
                    if j != curr_c and is_valid_move:
                        if (curr_r, j) not in path or (curr_r, j) == start_pos:
                            result = dfs(curr_r, j, path.copy(), 1)
                            if result: return result

            else: # Search Vertically (columns)
                for i in range(self.m):
                    is_valid_move = self.basis[i, curr_c] or (i, curr_c) == start_pos
                    if i != curr_r and is_valid_move:
                        if (i, curr_c) not in path or (i, curr_c) == start_pos:
                            result = dfs(i, curr_c, path.copy(), 0)
                            if result: return result

            return None

        cycle = dfs(start_r, start_c, [], 0)

        if cycle and cycle[0] == cycle[-1]:
            return cycle[:-1]

        return cycle

    def solve(self):

        # The main Transportation Simplex loop.

        start_time = time.time()

        self._get_initial_solution_nwc()

        while True:
            self.iterations += 1

            # 2. Calculate Potentials
            u, v = self._calculate_potentials()

            # 3. Calculate Reduced Costs and check optimality
            reduced_costs = self.costs - u[:, np.newaxis] - v
            reduced_costs[self.basis] = 0.0

            min_rc = reduced_costs.min()

            # Optimality check
            if min_rc >= -1e-6:
                break

            # Find the Entering Variable
            r_in, c_in = np.unravel_index(reduced_costs.argmin(), reduced_costs.shape)
            entering_node = (r_in, c_in)

            # 4. Find Cycle
            cycle = self._find_cycle(entering_node)

            if cycle is None:
                # Stops if severe degeneracy prevents cycle formation
                break

            self.chain_lengths.append(len(cycle))

            # 5. Determine Theta and Leaving Variable
            minus_nodes = cycle[1::2]
            allocations_at_minus = [self.allocation[r, c] for r, c in minus_nodes]
            if not allocations_at_minus: break

            theta = min(allocations_at_minus)

            # Identify Leaving Variable using strict float tolerance
            leaving_node = [node for node in minus_nodes if abs(self.allocation[node] - theta) < 1e-9][0]

            # 6. Update Allocation and Basis
            for idx, (r, c) in enumerate(cycle):
                if idx % 2 == 0:
                    self.allocation[r, c] += theta
                else:
                    self.allocation[r, c] -= theta

            self.basis[leaving_node] = False
            self.basis[entering_node] = True

            # Ensure the leaving cell is exactly zero
            self.allocation[leaving_node] = 0.0

        end_time = time.time()

        # Final Cost Calculation
        total_cost = np.sum(self.allocation * np.where(self.costs >= BIG_M, 0, self.costs))
        unfiltered_cost = np.sum(self.allocation * self.costs)

        # Final validity check: If Simplex used an M-value route, return BIG_M to ensure 0/10 fail
        if abs(total_cost - unfiltered_cost) > 1e-6:
             total_cost = BIG_M


        return {
            "status": "Optimal",
            "min_cost": total_cost,
            "iterations": self.iterations,
            "execution_time": end_time - start_time,
            "avg_chain_size": np.mean(self.chain_lengths) if self.chain_lengths else 0
        }

### Block 3: Benchmark Execution Function

The final blocks execute the empirical analysis to validate correctness and measure efficiency across varying problem scales, from $5 \times 5$ up to $100 \times 100$.

The PuLP Match Rate achieved a $100\%$ success rate, providing definitive quantitative proof that the custom solver's final minimum cost is correct and aligns perfectly with the established theoretical optimum.

The efficiency metrics demonstrate the algorithm's performance: Average Iterations confirm that the number of steps required to reach the optimum scales favorably with problem size. The low Average Time (s) confirms its high computational efficiency. Finally, the reported Average Chain Size provides auxiliary data on the internal complexity of the cycle-finding subroutine, illustrating how the length of the critical closed loop increases as the matrix dimensions expand.

In [15]:
def benchmark_suite_final():

    sizes = [(5, 5), (10, 10), (20, 20), (50, 50), (100, 100)]
    all_results = []

    # Print Header
    header = f"{'Size (m x n)':<15} | {'Avg. Iter.':<12} | {'Avg. Time (s)':<14} | {'Avg. Chain Size':<18} | {'PuLP Match Rate'}"
    print("--- Transportation Simplex Performance Analysis (Final NWC Run) ---", flush=True)
    print(header, flush=True)
    print("-" * (len(header) + 2), flush=True)

    for m, n in sizes:

        if m >= 100:
            runs_per_size = 5
        else:
            runs_per_size = 10

        iter_stats, time_stats, chain_stats = [], [], []
        correct_count = 0

        if m >= 50:
            print(f"Starting {m}x{n} with {runs_per_size} runs...", flush=True)

        # Execution Loop
        for run in range(runs_per_size):

            supply, demand, costs = generate_random_problem(m, n)
            solver = TransportationSimplex(supply, demand, costs)
            my_res = solver.solve()

            # PuLP Solver (Oracle)
            try:
                pulp_cost = solve_with_pulp(supply, demand, costs)
            except:
                pulp_cost = np.inf

            # Comparison
            is_correct = abs(my_res['min_cost'] - pulp_cost) < 1e-4
            if is_correct:
                correct_count += 1

            iter_stats.append(my_res['iterations'])
            time_stats.append(my_res['execution_time'])
            chain_stats.append(my_res['avg_chain_size'])

            # Print immediate progress for long runs
            if m == 100:
                print(f"  -> {m}x{n} Progress: {run+1}/{runs_per_size} finished.", flush=True)

        # Summarize and Print Result ROW (Forces Output)

        avg_iter = np.mean(iter_stats)
        avg_time = np.mean(time_stats)
        avg_chain = np.mean(chain_stats)

        summary_row = f"{m}x{n:<12} | {avg_iter:<12.1f} | {avg_time:<14.4f} | {avg_chain:<18.2f} | {correct_count}/{runs_per_size}"
        print(summary_row, flush=True)

        all_results.append({
            "size": f"{m}x{n}", "avg_iter": avg_iter, "avg_time": avg_time,
            "avg_chain": avg_chain, "match_rate": f"{correct_count}/{runs_per_size}"
        })

    return pd.DataFrame(all_results)


### Block 4: Final Execution

In [16]:
df_analysis = benchmark_suite_final()
print("\n--- Summary of Performance Results (DataFrame) ---", flush=True)
display(df_analysis)

--- Transportation Simplex Performance Analysis (Final NWC Run) ---
Size (m x n)    | Avg. Iter.   | Avg. Time (s)  | Avg. Chain Size    | PuLP Match Rate
----------------------------------------------------------------------------------------
5x5            | 6.0          | 0.0004         | 5.25               | 10/10
10x10           | 23.3         | 0.0021         | 7.78               | 10/10
20x20           | 67.5         | 0.0112         | 12.20              | 10/10
Starting 50x50 with 10 runs...
50x50           | 274.0        | 0.1529         | 24.94              | 10/10
Starting 100x100 with 5 runs...
  -> 100x100 Progress: 1/5 finished.
  -> 100x100 Progress: 2/5 finished.
  -> 100x100 Progress: 3/5 finished.
  -> 100x100 Progress: 4/5 finished.
  -> 100x100 Progress: 5/5 finished.
100x100          | 735.0        | 1.4650         | 47.16              | 5/5

--- Summary of Performance Results (DataFrame) ---


,size,avg_iter,avg_time,avg_chain,match_rate
0,5x5,6.0,0.000356,5.249524,10/10
1,10x10,23.3,0.002099,7.783217,10/10
2,20x20,67.5,0.011153,12.203736,10/10
3,50x50,274.0,0.152941,24.939354,10/10
4,100x100,735.0,1.465018,47.162079,5/5
